In [1]:
#####################################################################
## Clear all variables in the memory
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

In [7]:
#####################################################################
## Import retuired packages
import numpy as np
import pandas as pd
import mat73
import random
import calendar
import tensorflow as tf
import gc
from scipy.stats import mstats
from scipy.stats import rankdata
import matplotlib.pyplot as plt
import statsmodels.api as sm
import multiprocessing as mp
from multiprocessing import Pool
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
# import shap

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.python.keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
from keras.models import load_model

import eli5
from eli5.sklearn import PermutationImportance

import math
import openpyxl
import xlsxwriter
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')



In [2]:
#####################################################################
## Settings for the calculations that can be changed
########################################################################

Lags_Acc = 3

CurrentMonth = "202306"
PreviousMonth = "202305"

In [4]:
########################################################################
## Get Data
########################################################################

## =====================================================================
## I. Market Macro Data - Monthly
## ---------------------------------------------------------------------

Macro_Variables = pd.read_excel(r"C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\LLM_based_investment_research\Korean_Stock_Data\시장경제데이터_월별_"+CurrentMonth+".xlsx", index_col=0, header=0, sheet_name='시장데이터')
# Macro_Variables = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data"+CurrentMonth+"\\시장경제데이터_월별_"+CurrentMonth+".xlsx", index_col=0, header=0, sheet_name='시장데이터')


Dates_N = pd.to_numeric(Macro_Variables.index, errors='coerce')
Dates = Macro_Variables.index


## ---------------------------------------------------------------------
# The Number of Months
## ---------------------------------------------------------------------

T = len(Macro_Variables)


## ---------------------------------------------------------------------
# Treatment of Market-wide Variables
## ---------------------------------------------------------------------

RET_Mkt = (Macro_Variables['KOSPI'].astype('float32')-Macro_Variables['KOSPI'].astype('float32').shift(1))/Macro_Variables['KOSPI'].astype('float32').shift(1)*100
RET_KOSDAQ150 = (Macro_Variables['KOSDAQ150'].astype('float32')-Macro_Variables['KOSDAQ150'].astype('float32').shift(1))/Macro_Variables['KOSDAQ150'].astype('float32').shift(1)*100
CS = Macro_Variables['BBB-'].astype('float32') - Macro_Variables['AA-'].astype('float32')
TS = Macro_Variables['TB10Y'].astype('float32') - Macro_Variables['TB1Y'].astype('float32')
FR_Change = (Macro_Variables['Foreign_Reserve'].astype('float32').shift(1)-Macro_Variables['Foreign_Reserve'].astype('float32').shift(2))/Macro_Variables['Foreign_Reserve'].astype('float32').shift(2)*100  # Foreign reserve chagne
WTI = (Macro_Variables['WTI'].astype('float32')-Macro_Variables['WTI'].astype('float32').shift(1))/Macro_Variables['WTI'].astype('float32').shift(1)*100 
SP500 = (Macro_Variables['SP500'].astype('float32')-Macro_Variables['SP500'].astype('float32').shift(1))/Macro_Variables['SP500'].astype('float32').shift(1)*100   # S&P500 return
CN = (Macro_Variables['CN'].astype('float32')-Macro_Variables['CN'].astype('float32').shift(1))/Macro_Variables['CN'].astype('float32').shift(1)*100   # Shanghai Index return
IPI = (Macro_Variables['IPI'].astype('float32').shift(1)-Macro_Variables['IPI'].astype('float32').shift(2))/Macro_Variables['IPI'].astype('float32').shift(2)*100   # Industrial Production, one month lagged
CD = Macro_Variables['CD'].astype('float32')  # CD rate
KRW = (Macro_Variables['KRW'].astype('float32')-Macro_Variables['KRW'].astype('float32').shift(1))/Macro_Variables['KRW'].astype('float32').shift(1)*100
KVIX = Macro_Variables['KVIX'].astype('float32')  # CD rate



In [5]:
## =====================================================================
## II-1. Individual Stock Market Data - Monthly
## ---------------------------------------------------------------------


## ---------------------------------------------------------------------
## Reading Data
## ---------------------------------------------------------------------

FC_M = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data\개별기업시장데이터_월별_"+CurrentMonth+".xlsx", index_col=0, header=0, sheet_name = None)
# FC_M = pd.read_excel("D:\\Qsync\\consulting\\FnGuide\\Stock_Scoring_Project\\Crux_Trading\\data"+CurrentMonth+"\\개별기업시장데이터_월별_"+CurrentMonth+".xlsx", index_col=0, header=0, sheet_name = None)
FC_M_Variable_Names = list(FC_M.keys())   # names of variables


## ---------------------------------------------------------------------
## Calculation of Returns
## ---------------------------------------------------------------------

ret = FC_M['ret'].astype('float32')     # in percent
Dates_ret = ret.index.astype("int32")
Firm_Names = ret.columns
if np.array_equal(Dates,Dates_ret):
    ret = ret/100    # in decimal


## ---------------------------------------------------------------------
## The Number of Stocks
## ---------------------------------------------------------------------

N = len(ret.T)


## ---------------------------------------------------------------------
## Entire Individual Stock Market Data in the form of T by N  -  remioving the first column (dates)
## ---------------------------------------------------------------------

for fc_m in FC_M_Variable_Names:
    if fc_m not in ['mkt', 'fgsc']:
        Ind_Var = FC_M[fc_m].astype('float32')
        Ind_Var_Dates = Ind_Var.index
        Ind_Var_IDs = Ind_Var.columns
        if not np.array_equal(Dates,Ind_Var_Dates):
            print([fc_m+" : Dates of Macro Data and Individual Stock Data Do Not Match!"])
        elif not np.array_equal(Firm_Names,Ind_Var_IDs):
            print([fc_m+" : Firms of Return Data and Firm Characteristics Data Do Not Match!"])
        else:
            FC_M[fc_m] = Ind_Var
            FC_M[fc_m].index = Dates
            FC_M[fc_m].columns = Firm_Names
    elif fc_m in ['mkt', 'fgsc']:
        Ind_Var = FC_M[fc_m]
        Ind_Var_Dates = Ind_Var.index
        Ind_Var_IDs = Ind_Var.columns
        if not np.array_equal(Dates,Ind_Var_Dates):
            print([fc_m+" : Dates of Macro Data and Individual Stock Data Do Not Match!"])
        elif not np.array_equal(Firm_Names,Ind_Var_IDs):
            print([fc_m+" : Firms of Return Data and Firm Characteristics Data Do Not Match!"])
        else:
            FC_M[fc_m] = Ind_Var
            FC_M[fc_m].index = Dates
            FC_M[fc_m].columns = Firm_Names            


## ---------------------------------------------------------------------
## Variables for the Prediction of Returns
## ---------------------------------------------------------------------

FC_M['trad_ind'] = pd.DataFrame(FC_M['trad_ind']/FC_M['tvolm']*10000/100, index=Dates, columns=Firm_Names)    # in percent
# FC_M['trad_ind'].index = Dates

FC_M['trad_institution'] = pd.DataFrame(FC_M['trad_institution']/FC_M['tvolm']*10000/100, index=Dates, columns=Firm_Names)    # in percent
# FC_M['trad_institution'].index = Dates

FC_M['trad_foreign'] = pd.DataFrame(FC_M['trad_foreign']/FC_M['tvolm']*10000/100, index=Dates, columns=Firm_Names)    # in percent
# FC_M['trad_foreign'].index = Dates

TVOLM = pd.DataFrame(FC_M['tvolm'], index=Dates, columns=Firm_Names) 
FC_M['tvolm'] = pd.DataFrame(FC_M['tvolm']/FC_M['m_value']/1000000*100, index=Dates, columns=Firm_Names)  # in percent
# FC_M['tvolm'].index = Dates

FC_M['short_int'] = pd.DataFrame(FC_M['short_int']/FC_M['shrout']*100, index=Dates, columns=Firm_Names)   # in percent
# FC_M['short_int'].index = Dates

# Age (years) since IPO in years
TT1 = np.trunc(FC_M['ipo_date']/10000)
TT3 = np.trunc((Dates_N.astype(float))/100)
TT2 = np.empty((T,N),)
for i in range(0,N):
    TT2[:,i] = TT3 - TT1.iloc[:,i]
FC_M['ipo_date'] = pd.DataFrame(np.where(TT2 <= 0, np.nan, TT2), index=Dates, columns=Firm_Names)    
FC_M['age'] = FC_M.pop('ipo_date')

FC_M['m_value'] = pd.DataFrame(FC_M['m_value'], index=Dates, columns=Firm_Names)
# FC_M['m_value'].index = Dates
M_Value = FC_M['m_value'] 

FC_M['anal_rating'] = pd.DataFrame(FC_M['anal_rating'], index=Dates, columns=Firm_Names)
# FC_M['anal_rating'].index = Dates 


## ---------------------------------------------------------------------
## Saved for other calculations
## ---------------------------------------------------------------------

# nominal price
FC_M['prc'] = FC_M.pop('price')

# 1: KSE, 2: KOSDAQ
TT1 = np.where(FC_M['mkt']=='KSE', 1, 0) + np.where(FC_M['mkt']=='KOSDAQ', 2, 0)  
EXCH = pd.DataFrame(np.where(TT1==0, np.nan, TT1), index=Dates, columns=Firm_Names)     

# Sector Classification
TT1 = np.zeros((T,N))
for i in range(0,N):
    for t in range(0,T):
        if not pd.isna(FC_M['fgsc'].iloc[t,i]):
            TT2 = FC_M['fgsc'].iloc[t,i].split('.')
            TT1[t,i] = TT2[1]                            # first 2 digits
FGSC = pd.DataFrame(np.where(TT1 > 0, TT1, np.nan), index=Dates, columns=Firm_Names)


# ## ---------------------------------------------------------------------
# ## Remove Variables in FC_M that are not used for the prediction of returns
# ## ---------------------------------------------------------------------

for fc_m in list(FC_M):
    if fc_m in ['ret', 'mkt', 'acc_year', 'fgsc']:
        del FC_M[fc_m]


## ---------------------------------------------------------------------
## Saved Firm Characteristics Variables for the prediction of returns
## ---------------------------------------------------------------------

Items = list(FC_M.keys())
for k in Items:
    if k not in ['age', 'prc', 'exch', 'm_value']:
        FC_M[k+'_D'] = FC_M[k] - FC_M[k].shift(1)


## 'm_value'는 금액단위로 되어 있음
FC_M['m_value_D'] = np.log1p(FC_M['m_value']) - np.log1p(FC_M['m_value'].shift(1))
FC_M['m_value'] = np.log1p(FC_M['m_value'])


print(FC_M.keys())

del Ind_Var, Ind_Var_Dates, Ind_Var_IDs, FC_M_Variable_Names, TT1, TT2, TT3

dict_keys(['shrout', 'm_value', 'short_int', 'anal_rating', 'trad_institution', 'trad_ind', 'trad_foreign', 'tvolm', 'age', 'prc', 'shrout_D', 'short_int_D', 'anal_rating_D', 'trad_institution_D', 'trad_ind_D', 'trad_foreign_D', 'tvolm_D', 'm_value_D'])


In [6]:
## =====================================================================
## III. Individual Stock FC and Accounting Data - Monthly
## ---------------------------------------------------------------------

##  FC_M = 'tvolm', 'm_value', 'short_int', 'anal_rating', 'trad_institution', 'trad_ind', 'trad_foreign', 'age', 'prc'


## Accounting Items Classification
Acc_Classification = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data\Acc_Items_Names.xlsx", sheet_name = None, header=None, index_col=None)
# list(Acc_Classification.keys())
Sub_set_Accs = ['Acc_Items_Names','Acc_BS', 'Acc_IS', 'Acc_PerShare']

Accs_BSIS_Items = {}
for k in range(0, len(Sub_set_Accs)):
    ttt = []
    for j in range(0,len(Acc_Classification[Sub_set_Accs[k]])):
        ttt.append(Acc_Classification[Sub_set_Accs[k]][0].values.tolist())
    Accs_BSIS_Items[Sub_set_Accs[k]] = ttt[0]
 
## Reading Accounting Variables
Acc_Items = {}
for k in range(1,11):
    # Acc_Items_Original = pd.read_excel("D:\\Qsync\\consulting\\FnGuide\\Stock_Scoring_Project\\Crux_Trading\\data"+CurrentMonth+"\\Acc_Items"+str(k)+"_"+CurrentMonth+".xlsx", sheet_name = None, header=0, index_col=0)
    Acc_Items_Original = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data\Acc_Items"+str(k)+"_"+CurrentMonth+".xlsx", sheet_name = None, header=0, index_col=0)
    Acc_Items_Names = list(Acc_Items_Original.keys())
    for m in Acc_Items_Names:
        Acc_Items[m] = Acc_Items_Original[m]

## Standardizing Accounting Variables by Sales or Assets
Acc_Items_D = {}
Acc_Items_M = {}
Acc_Items_M_D = {}
Acc_Items_Names  = list(Acc_Items.keys())
for k in Acc_Items_Names:
    # print(k, Acc_Items[k].notnull().sum().sum())
    if k in Accs_BSIS_Items['Acc_BS'] and k != '자산':
        Acc_Items[k] = (Acc_Items[k].shift(12+Lags_Acc)/Acc_Items['자산'].shift(12+Lags_Acc)).replace([np.inf, -np.inf], np.nan)
        Acc_Items_D[k+'_D'] = Acc_Items[k] - Acc_Items[k].shift(12)
        Acc_Items_M[k+'_M'] = (Acc_Items[k]*Acc_Items['자산'].shift(12+Lags_Acc)/M_Value/1000).replace([np.inf, -np.inf], np.nan)
        Acc_Items_M_D[k+'_M_D'] =  Acc_Items_M[k+'_M'] - Acc_Items_M[k+'_M'].shift(12)
    elif k in Accs_BSIS_Items['Acc_IS'] and k != '매출액':
        Acc_Items[k] = (Acc_Items[k].shift(12+Lags_Acc)/Acc_Items['매출액'].shift(12+Lags_Acc)).replace([np.inf, -np.inf], np.nan)
        Acc_Items_D[k+'_D'] = Acc_Items[k] - Acc_Items[k].shift(12)
        Acc_Items_M[k+'_M'] = (Acc_Items[k]*Acc_Items['매출액'].shift(12+Lags_Acc)/M_Value/1000).replace([np.inf, -np.inf], np.nan)
        Acc_Items_M_D[k+'_M_D'] =  Acc_Items_M[k+'_M'] - Acc_Items_M[k+'_M'].shift(12)
    else:
        Acc_Items[k] = Acc_Items[k].shift(12+Lags_Acc)
        Acc_Items_D[k+'_D'] = Acc_Items[k] - Acc_Items[k].shift(12)

Acc_Items['매출액_자산'] = Acc_Items['매출액'].shift(12+Lags_Acc)/Acc_Items['자산'].shift(12+Lags_Acc)        
Acc_Items['매출액_자산_D'] = Acc_Items['매출액_자산'] -Acc_Items['매출액_자산'].shift(12)
Acc_Items['자산_D'] = np.log1p(Acc_Items['자산'].shift(12+Lags_Acc)) - np.log1p(Acc_Items['자산'].shift(12+12+Lags_Acc))
Acc_Items['매출액_D'] = np.log1p(Acc_Items['매출액'].shift(12+Lags_Acc)) - np.log1p(Acc_Items['매출액'].shift(12+12+Lags_Acc))
Acc_Items['자산_M'] = (Acc_Items['자산'].shift(12+Lags_Acc)/M_Value/1000).replace([np.inf, -np.inf], np.nan)
Acc_Items['매출액_M'] = (Acc_Items['매출액'].shift(12+Lags_Acc)/M_Value/1000).replace([np.inf, -np.inf], np.nan)
Acc_Items['자산_M_D'] = Acc_Items['자산_M'] - Acc_Items['자산_M'].shift(12+Lags_Acc)
Acc_Items['매출액_M_D'] = Acc_Items['매출액_M'] - Acc_Items['매출액_M'].shift(12+Lags_Acc)
Acc_Items['자산'] = np.log1p(Acc_Items['자산'])
Acc_Items['매출액'] = np.log1p(Acc_Items['매출액'])

FC_M.update(Acc_Items) 
FC_M.update(Acc_Items_D) 
FC_M.update(Acc_Items_M) 
FC_M.update(Acc_Items_M_D) 

## Accounting Variables : Precalculated
# FC_AA_Items = pd.read_excel("D:\\Qsync\\consulting\\FnGuide\\Stock_Scoring_Project\\Crux_Trading\\data"+CurrentMonth+"\\FC_AA_"+CurrentMonth+'.xlsx', sheet_name = None, index_col=0)
FC_AA_Items = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data\FC_AA_"+CurrentMonth+'.xlsx', sheet_name = None, index_col=0)
Items = list(FC_AA_Items.keys())
for k in Items:
    # print(k, FC_AA_Items[k].notnull().sum().sum())
    FC_AA_Items[k] = FC_AA_Items[k].shift(12+Lags_Acc)
    FC_AA_Items[k+'_D'] = FC_AA_Items[k] - FC_AA_Items[k].shift(12)
FC_M.update(FC_AA_Items) 

## Accounting + Market Variables : Precalculated
# FC_AM_Items = pd.read_excel("D:\\Qsync\\consulting\\FnGuide\\Stock_Scoring_Project\\Crux_Trading\\data"+CurrentMonth+"\\FC_AM_"+CurrentMonth+'.xlsx', sheet_name = None, index_col=0)
FC_AM_Items = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data\FC_AM_"+CurrentMonth+'.xlsx', sheet_name = None, index_col=0)
Items = list(FC_AM_Items.keys())
for k in Items:
    # print(k, FC_AM_Items[k].notnull().sum().sum())
    FC_AM_Items[k] = FC_AM_Items[k]
    FC_AM_Items[k+'_D'] = FC_AM_Items[k] - FC_AM_Items[k].shift(1)
FC_M.update(FC_AM_Items) 
 
## Market Variables : Precalculated
# FC_MM_Items = pd.read_excel("D:\\Qsync\\consulting\\FnGuide\\Stock_Scoring_Project\\Crux_Trading\\data"+CurrentMonth+"\\FC_MM_"+CurrentMonth+'.xlsx', sheet_name = None, index_col=0)
FC_MM_Items = pd.read_excel(r"C:\Users\boyu571\Python\LLM_based_investment_research\Korean_Stock_Data\FC_MM_"+CurrentMonth+'.xlsx', sheet_name = None, index_col=0)
Items = list(FC_MM_Items.keys())
for k in Items:
    # print(k, FC_MM_Items[k].notnull().sum().sum())
    FC_MM_Items[k] = FC_MM_Items[k]
    FC_MM_Items[k+'_D'] = FC_MM_Items[k] - FC_MM_Items[k].shift(1)
FC_M.update(FC_MM_Items) 

fnames_all=list(FC_M.keys())

Beta_KOSPI = pd.DataFrame(FC_M['Beta_KOSPI'].astype('float32'), index=Dates, columns=Firm_Names) 
Beta_KOSPI200 = pd.DataFrame(FC_M['Beta_KOSPI200'].astype('float32'), index=Dates, columns=Firm_Names) 
Beta_KOSDAQ = pd.DataFrame(FC_M['Beta_KOSDAQ'].astype('float32'), index=Dates, columns=Firm_Names) 
Beta_KOSDAQ150 = pd.DataFrame(FC_M['Beta_KOSDAQ150'].astype('float32'), index=Dates, columns=Firm_Names) 

del Acc_Items_Original, Acc_Classification, Accs_BSIS_Items, Acc_Items, Acc_Items_D, Acc_Items_M, Acc_Items_M_D, FC_AA_Items, FC_AM_Items, FC_MM_Items, Items


In [7]:
FC_M_Names = list(FC_M.keys())
print(len(FC_M_Names))
print(FC_M_Names)
for k in FC_M_Names:
    print(FC_M[k].index[0], FC_M[k].columns[0], k)

758
['shrout', 'm_value', 'short_int', 'anal_rating', 'trad_institution', 'trad_ind', 'trad_foreign', 'tvolm', 'age', 'prc', 'shrout_D', 'short_int_D', 'anal_rating_D', 'trad_institution_D', 'trad_ind_D', 'trad_foreign_D', 'tvolm_D', 'm_value_D', 'BIS비율', 'EBITDA_이자비용', 'EPS', 'EVA', 'PBR', 'PCR', 'PEGR', 'PER', 'ROA(영업이익)', 'ROE(영업이익)', 'ROIC', 'WACC', '감가상각비', '감채적립금', '경과손해율', '경영지원자산', '계약자지분조정', '관계기업투자등관련손익(비영업)', '관리비', '광고선전비', '금융상품관련손실(영업)', '금융수익', '금융원가', '금융자산관련손익', '기업구조조정자산', '기타예수유가증권', '기타유형자산', '기타포괄손익누계액', '단기사채', '단기차입금', '담보대출금', '당기순손익', '당기순이익(손실)', '당좌비율', '대손상각비(판관비)', '대손충당금', '대출채권', '대출채권_자본금', '대출채권이자수익', '대출채권평균이자율', '리스및렌탈자산', '매입채무및기타유동채무', '매입채무평균지급기간', '매입채무회전율', '매출액', '매출원가', '매출채권', '매출채권_매입채무', '매출채권평균회수기간', '매출채권회전율', '매출총이익(손실)', '무형자산', '무형자산구성비율', '무형자산상각률', '무형자산상각비', '미수금', '발행어음예수금', '배당수익률(보통주,현금)', '법인세비용', '법인세비용차감전계속사업이익', '베타', '보통주자본금', '보험계약준비금_총자본', '보험료수익', '보험약관대출금', '복리후생비', '부가가치', '부채', '비유동부채', '비유동부채비율', '비유동자산', '비유동장기적합률', '

In [10]:
ret

,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,A000110,...,A950110,A950130,A950140,A950160,A950170,A950180,A950190,A950200,A950210,A950220
200001,0.0037,-0.2409,0.3608,-0.0277,0.4127,0.0949,0.0305,NaN,-0.0665,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200002,0.0346,-0.3139,-0.0909,0.0166,-0.1507,0.1889,0.0000,NaN,0.1888,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200003,0.0565,-0.1350,0.0667,0.1943,-0.0847,-0.1854,-0.1531,NaN,-0.0529,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200004,-0.2550,-0.0260,-0.0312,-0.1274,-0.3366,0.0732,-0.1254,NaN,-0.1498,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200005,-0.2095,0.1246,-0.2016,-0.0911,0.0378,-0.2614,0.0333,NaN,0.0312,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202302,-0.0384,NaN,-0.0358,0.0262,0.0000,0.0960,-0.0913,NaN,0.0076,NaN,...,0.0992,-0.0629,-0.0545,-0.0053,0.0824,NaN,-0.0078,-0.0697,0.2557,-0.0541
202303,-0.0787,NaN,-0.0156,-0.0246,NaN,-0.0444,-0.0628,NaN,-0.0398,NaN,...,0.1383,-0.1214,-0.0535,-0.0063,-0.0938,NaN,-0.0369,-0.0499,0.0841,-0.0948
202304,0.0084,NaN,0.3294,-0.0140,NaN,0.0040,-0.0067,NaN,0.1383,NaN,...,-0.0542,0.0640,-0.0239,0.1608,0.0706,NaN,0.0652,0.0680,-0.1235,-0.0200
202305,0.1456,NaN,0.0000,-0.0180,NaN,-0.0661,0.0292,NaN,0.0295,NaN,...,0.0722,-0.0433,0.8441,-0.1138,-0.1047,NaN,-0.0704,0.0318,0.0101,-0.0629


In [8]:
ret.to_csv('수익률.csv',encoding='utf-8')

In [14]:
Beta_KOSPI['A005930']

200001         NaN
200002         NaN
200003         NaN
200004         NaN
200005         NaN
            ...   
202302    1.057133
202303    1.062106
202304    1.071522
202305    1.089906
202306    1.131959
Name: A005930, Length: 282, dtype: float32

In [17]:
FC_M['매출총이익(손실)']

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A950110,A950130,A950140,A950160,A950170,A950180,A950190,A950200,A950210,A950220
200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202302,NaN,0.548918,NaN,0.150064,0.414870,NaN,0.295008,0.410595,NaN,0.323885,...,0.721133,3.254495,0.251517,0.123257,-0.458286,0.877156,0.587333,0.308768,NaN,NaN
202303,NaN,0.548918,NaN,0.150064,0.414870,NaN,0.295008,0.410595,NaN,0.323885,...,0.721133,3.254495,0.251517,0.123257,-0.458286,0.877156,0.587333,0.308768,NaN,NaN
202304,NaN,0.665221,NaN,0.144529,0.383288,NaN,0.219855,0.471226,NaN,0.319904,...,0.721133,6.730577,0.188481,2.162926,-0.458286,0.877156,0.578275,0.635266,NaN,NaN
202305,NaN,0.665221,NaN,0.144529,0.383288,NaN,0.219855,0.471226,NaN,0.319904,...,0.721133,6.730577,0.188481,2.162926,-0.458286,0.877156,0.578275,0.635266,NaN,NaN


In [12]:
PBR.to_csv('PBR')

In [13]:
당기순손익 = pd.DataFrame(FC_M['당기순손익'].astype('float32'), index=Dates, columns=Firm_Names) 
당기순손익.to_csv('당기순손익')

In [9]:
FGSC

,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,A000110,...,A950110,A950130,A950140,A950160,A950170,A950180,A950190,A950200,A950210,A950220
200001,35.0,40.0,25.0,25.0,40.0,25.0,20.0,NaN,35.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200002,35.0,40.0,25.0,25.0,40.0,25.0,20.0,NaN,35.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200003,35.0,40.0,25.0,25.0,40.0,25.0,20.0,NaN,35.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200004,35.0,40.0,25.0,25.0,40.0,25.0,20.0,NaN,35.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200005,35.0,40.0,25.0,25.0,40.0,25.0,20.0,NaN,35.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202302,35.0,NaN,25.0,25.0,40.0,15.0,30.0,NaN,35.0,NaN,...,45.0,35.0,30.0,35.0,25.0,NaN,45.0,35.0,35.0,35.0
202303,35.0,NaN,25.0,25.0,40.0,15.0,30.0,NaN,35.0,NaN,...,45.0,35.0,30.0,35.0,25.0,NaN,45.0,35.0,35.0,35.0
202304,35.0,NaN,25.0,25.0,NaN,15.0,30.0,NaN,35.0,NaN,...,45.0,35.0,30.0,35.0,25.0,NaN,45.0,35.0,35.0,35.0
202305,35.0,NaN,25.0,25.0,NaN,15.0,30.0,NaN,35.0,NaN,...,45.0,35.0,30.0,35.0,25.0,NaN,45.0,35.0,35.0,35.0


In [17]:
pd.set_option("display.max_rows", None)

In [9]:
daily_a = pd.read_excel(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\LLM_based_investment_research\Korean_Stock_Data\개별기업시장데이터A_일별_202306.xlsx')

In [13]:
ret_daily = daily_a

In [15]:
ret_daily.to_csv("일별 수익률.csv")

In [18]:
ret["A000660"]

200001    0.0954
200002   -0.3220
200003    0.3743
200004   -0.2846
200005    0.0000
200006    0.2500
200007   -0.2091
200008    0.1667
200009   -0.2143
200010   -0.5618
200011   -0.0215
200012   -0.4116
200101    0.4534
200102   -0.3590
200103   -0.1587
200104    0.0729
200105    0.2821
200106   -0.3560
200107   -0.5080
200108   -0.3964
200109    0.0662
200110    0.2599
200111    0.9238
200112    0.1282
200201    0.0083
200202   -0.3381
200203   -0.1084
200204   -0.3368
200205   -0.2775
200206   -0.6232
200207    0.9614
200208    0.0490
200209   -0.2056
200210    0.1176
200211   -0.0842
200212   -0.3564
200301   -0.1607
200302   -0.1489
200303   -0.3250
200304    0.2189
200305    0.1811
200306    0.5177
200307    0.2763
200308    0.0785
200309   -0.1581
200310    0.1222
200311   -0.1195
200312   -0.1554
200401    0.5982
200402    0.0022
200403    0.3657
200404   -0.0286
200405   -0.0546
200406    0.1244
200407   -0.2885
200408    0.1611
200409    0.0048
200410    0.2095
200411   -0.18